In [26]:
import pandas as pd
import numpy as np
import xlsxwriter
from datetime import datetime
from xlsxwriter.utility import xl_rowcol_to_cell, xl_cell_to_rowcol

# Setup

## set up workbook 

In [27]:
workbook = xlsxwriter.Workbook('IslandEnergy.xlsx')
ws_const_in = workbook.add_worksheet(name="Construction Period Cost Inputs")
ws_const_calc = workbook.add_worksheet(name="Construction Period Calcs")

# dictionary of references 
refd= {}
refd["sheetnames"] = list(workbook.sheetnames.keys())
refd["gen_startrow"] = 4 

## formatting 

In [28]:
# Add a bold format to use to highlight cells.
bold_format = workbook.add_format({'bold': True})

# Add a number format for cells with money.
money_format = workbook.add_format({'num_format': '$#,##0'})

# create format to use in merged range 
merge_format = workbook.add_format({
    'bold': 1,
    # 'border': 1,
    # 'align': 'center',
    # 'valign': 'vcenter',
    # 'fg_color': 'yellow'
    })

## worksheet formatting 

In [29]:
# Page Headers 
for name, sheet in workbook.sheetnames.items():
    sheet.write('A1', 'Project Name', bold_format)
    sheet.write('A2', 'Sheet Name', bold_format)
    sheet.merge_range('B1:C1', "Island Energy", bold_format)
    sheet.merge_range('B2:C2', name, bold_format)
    sheet.set_column(0, 30, 20)

# Inputs 

## construction period inputs

In [30]:
# data 
# payments at financial close  
# TODO put these in a dif file 
fc_costs = {
    "Land": 12.5e6,
    "Development Expenses": 28.5e6,
    "Development Fees": 14.7e6,
    "Closing Costs" : 14.392e6,
    "Contingency Fees": 30.754e6
}

# eng, procurement, const base cost 
epc_cost = 455e6

# other payments over course of construction period 
split_costs = {
"Interest and Fees" : 56.853e6,
"Insurance and Fees" : 25e6,
"Management and Oversight ": 8.15e6
}

In [31]:
# costs paid at financial close 
ws_const_in.merge_range('A4:B4', 'Costs Paid at Financial Close', merge_format)

refd["cpi"] = {}
refd["cpi"]["heading_row"] = xl_cell_to_rowcol('A4') 


row = refd["cpi"]["heading_row"][0]+1
refd["cpi"]["fc_close_sr"] = row
col = 0
for k,v in fc_costs.items():
    ws_const_in.write(row, col, k)
    ws_const_in.write(row, col+1, v, money_format)
    row += 1



In [32]:
# costs paid over construction period 

# skip a row to make header 
row+=1
ws_const_in.merge_range(row, 0, row, 1, 'Costs Paid Over Construction Period', merge_format)

row+=1
ws_const_in.write(row, col, "EPC Cost")
ws_const_in.write(row, col+1, epc_cost)

row+=1
refd["cpi"]["split_sr"] = row
for k,v in split_costs.items():
    ws_const_in.write(row, col, k)
    ws_const_in.write(row, col+1, v, money_format)
    row += 1

# Calculations 

## construction calculations 

In [33]:
months = range(1,25)

In [34]:
# TODO: EPC payment schedule 

headings = ["Payment Schedule Category", "Payment Name", "Financial Close"] + list(months) # TOOD -> make this nice, month 1 + date 

col=0
for heading in headings:
    ws_const_calc.write(refd["gen_startrow"], col, heading, bold_format)
    col +=1


In [35]:
# # costs paid monthly - split costs 
# len_split_costs = len(list(split_costs.keys()))

# refd["cc"] = {}
# refd["cc"]["monthly_exp_sr"] = refd["gen_startrow"] + 1

# # TODO monthly costs merge in vertical 
# sheet.merge_range(refd["cc"]["monthly_exp_sr"], 0, refd["cc"]["monthly_exp_sr"]+len_split_costs-1, 0, "Monthly Split Costs", bold_format)

# # local columns
# name_col = 1
# eqn_start_col = 2
# # local row 
# row = refd["cc"]["monthly_exp_sr"]
# # for iterating over the foreign sheet
# i=0 

# for cost in range(len_split_costs):
#     # get cell name from other sheet 
#     name_cell = xl_rowcol_to_cell(refd["cpi"]["split_sr"]+i, 0)
#     # write heading 
#     ws_const_calc.write_formula(row, name_col, f"='{refd['sheetnames'][0]}'!{name_cell}")

#     # do monthly calculation 
#     for month in months:
#         # get cell name from other sheet 
#         data_cell = xl_rowcol_to_cell(refd["cpi"]["split_sr"]+i, 1)
#     # do calculation 
#         ws_const_calc.write_formula(row, eqn_start_col+month, f"='{refd['sheetnames'][0]}'!{data_cell}/24", money_format)

#     # iterate over foreign and local sheet 
#     row+=1
#     i+=1


In [36]:
def calc_on_inputs(local_start_row, foreign_start_row, input_data, merge_name):
    # costs paid at financial close 
    len_data = len(list(input_data.keys()))

    # documenting start and end of this area, not needed in this fx 
    # refd["cc"]["fc_sr"] = refd["cc"]["monthly_exp_sr"] + 1

    # TODO monthly costs merge in vertical 
    sheet.merge_range(local_start_row+1, 0, local_start_row+len_data, 0, merge_name, bold_format)

    # local columns
    name_col = 1
    eqn_start_col = 2
    # local row 
    row = local_start_row+1
    # for iterating over the foreign sheet
    i=0 

    for cost in range(len_data):
        # get cell name from other sheet 
        name_cell = xl_rowcol_to_cell(foreign_start_row+i, 0)
        # write heading 
        ws_const_calc.write_formula(row, name_col, f"='{refd['sheetnames'][0]}'!{name_cell}") #TODO change sheetname to be more broad later 

        # do monthly calculation 
        for month in months:
            # get cell name from other sheet 
            data_cell = xl_rowcol_to_cell(foreign_start_row+i, 1)
        # do calculation 
            ws_const_calc.write_formula(row, eqn_start_col+month, f"='{refd['sheetnames'][0]}'!{data_cell}/24", money_format)

        # iterate over foreign and local sheet 
        row+=1
        i+=1

    return row # return ending row 

In [37]:
end_row = calc_on_inputs(refd["gen_startrow"], refd["cpi"]["split_sr"], split_costs, "Monthly Payments")
end_row

8

In [38]:
# # costs paid at financial close 
# len_split_costs = len(list(fc_costs.keys()))

# refd["cc"] = {}
# refd["cc"]["fc_sr"] = refd["cc"]["monthly_exp_sr"] + 1

# # TODO monthly costs merge in vertical 
# sheet.merge_range(refd["cc"]["monthly_exp_sr"], 0, refd["cc"]["monthly_exp_sr"]+len_split_costs-1, 0, "Monthly Split Costs", bold_format)

# # local columns
# name_col = 1
# eqn_start_col = 2
# # local row 
# row = refd["cc"]["monthly_exp_sr"]
# # for iterating over the foreign sheet
# i=0 

# for cost in range(len_split_costs):
#     # get cell name from other sheet 
#     name_cell = xl_rowcol_to_cell(refd["cpi"]["split_sr"]+i, 0)
#     # write heading 
#     ws_const_calc.write_formula(row, name_col, f"='{refd['sheetnames'][0]}'!{name_cell}")

#     # do monthly calculation 
#     for month in months:
#         # get cell name from other sheet 
#         data_cell = xl_rowcol_to_cell(refd["cpi"]["split_sr"]+i, 1)
#     # do calculation 
#         ws_const_calc.write_formula(row, eqn_start_col+month, f"='{refd['sheetnames'][0]}'!{data_cell}/24", money_format)

#     # iterate over foreign and local sheet 
#     row+=1
#     i+=1

In [39]:
# set up headings 
# def const_calcs_sheet():


# Close

In [40]:
workbook.close()